# Эксперимент с наивным Байесом

In [ ]:
!pip install natasha

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

### Исправление опечаток

In [ ]:
!pip install pyaspeller

In [ ]:
# https://ru.stackoverflow.com/questions/1142246/yandex-spellchecker-%D0%BE%D1%82%D0%BA%D0%B0%D0%B7%D1%8B%D0%B2%D0%B0%D0%B5%D1%82%D1%81%D1%8F-%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D1%82%D1%8C-%D0%BD%D0%B0-%D1%81%D0%B8%D0%BB%D1%8C%D0%BD%D0%BE-%D0%B8%D1%81%D0%BA%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0%D0%BD%D0%BD%D0%BE%D0%BC-%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B5

from pyaspeller import YandexSpeller

def spellcheck(text):
    try:
        speller = YandexSpeller()
        changes = {change['word']: change['s'][0] for change in speller.spell(text)}
        for word, suggestion in changes.items():
            text = text.replace(word, suggestion)
        return text
    except:
        return 'error'


In [ ]:
spellcheck('лучьший')

'лучший'

### Создание ДФ с сущностями и контекстами

In [ ]:
import pandas as pd
from nltk.tokenize import sent_tokenize
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import nltk
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import warnings
from copy import deepcopy

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
ans = pd.read_csv('https://raw.githubusercontent.com/PhilBurub/NLPcourse_HSE/main/train_aspects.txt', header=None, delimiter='\t', index_col=0,
                  names=['aspect', 'entity', 'start', 'end', 'sentiment'])
texts = pd.read_csv('https://raw.githubusercontent.com/PhilBurub/NLPcourse_HSE/main/train_reviews.txt', header=None, delimiter='\t', index_col=0,
                    names=['text'])

In [ ]:
def boudaries(id_, text):
  out = []
  new_text = text
  cur = 0
  for sent in sent_tokenize(text):
    start = new_text.find(sent)
    end = start + len(sent)
    out.append((id_, sent, cur + start, cur + end))
    cur += end
    new_text = new_text[end:]
  return pd.DataFrame(out, columns=['id', 'text', 'start', 'end'])

In [ ]:
class ContextDataset(Dataset):
  def __init__(self, texts, outputs):

    new = pd.DataFrame()
    for id_, text in texts.iterrows():
      new = pd.concat((new, boudaries(id_, text['text'])))

    out = []
    for text_id, row in outputs.iterrows():
      slice_ = new[(new['id'] == text_id) & (new['start'] <= row['start']) & (new['end'] >= row['end'])]
      if len(slice_) == 0:
        print(row, '0')
        continue
      if len(slice_) > 1:
        print(row, '>1')
        continue
      out.append((text_id, row['entity'], row['aspect'], slice_['text'].item(), row['sentiment']))
    self.contexts = pd.DataFrame(out, columns=['id', 'input', 'aspect', 'context', 'sentiment'])

    self.map_sent = {0: -1, 'negative': 0, 'neutral': 1, 'positive': 2,
                     'both': 3}
    self.map_asp = {0: -1, 'Food': 0, 'Interior': 1, 'Price': 2,
                    'Service': 3, 'Whole': 4}

  def __len__(self):
    return len(self.contexts)

  def __getitem__(self, idx):
    if isinstance(idx, slice):
      one = False
    else:
      one = True

    row = self.contexts.loc[idx]
    with warnings.catch_warnings():
      warnings.simplefilter("ignore")
      if one:
        row.loc['CLS'] = '[CLS]'
      else:
        row.loc[:, 'CLS'] = '[CLS]'
    sent = row[['input', 'CLS', 'context']].values
    sentence = ' '.join(sent) if one else list(map(lambda x: ' '.join(x), sent))
    aspect = [self.map_asp.get(row['aspect'])] if one else \
                              row['aspect'].apply(self.map_asp.get).tolist()
    sentiment = [self.map_sent.get(row['sentiment'])] if one else \
                              row['sentiment'].apply(self.map_sent.get).tolist()
    return sentence, torch.tensor(aspect, dtype=torch.int32), torch.tensor(sentiment, dtype=torch.int32)

In [ ]:
train_set = ContextDataset(texts, ans.iloc[:4000])
test_set = ContextDataset(texts, ans.iloc[4000:])

Тональный словарь.

In [ ]:
kss = pd.read_csv('https://raw.githubusercontent.com/NuzhnenkoAleksandra/NLP-Course-4-Year-HSE/main/project/kartaslovsent.csv', header=0, delimiter=';', index_col=None)

In [ ]:
kss.term

0              абажур
1           аббатство
2        аббревиатура
3               абзац
4           абиссинец
             ...     
46122         ёмкость
46123          ёрзать
46124        ёрничать
46125             ёрш
46126           ёршик
Name: term, Length: 46127, dtype: object

In [ ]:
kss[kss['term'] == 'абажур']

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.58,0.198,0.0


In [ ]:
int(kss[kss['term'] == 'абажур'].value)

0

In [ ]:
kss_terms = list(kss['term'])
len(kss_terms)

46127

Удаляем ё в тональном словаре, т. к. Наташа лемматезирует без ё.

In [ ]:
def delete_yo(text):
    text = text.replace('ё', 'е')

    text = text.lower()

    return text

# замена текста с ё на текст с е в дф-тональном словаре
kss['term'] = kss['term'].apply(delete_yo)

In [ ]:
train_set.contexts

,id,input,aspect,context,sentiment
0,3976,ресторане,Whole,Решил написать отзыв о ресторане в котором отм...,neutral
1,3976,ресторанах,Whole,Решил написать отзыв о ресторане в котором отм...,neutral
2,3976,ресторане,Whole,Но теперь о ресторане.,neutral
3,3976,Столик бронировали,Service,Столик бронировали заранее и сделали так как п...,neutral
4,3976,администратор,Service,Столик бронировали заранее и сделали так как п...,positive
...,...,...,...,...,...
3995,15567,Официантка,Service,Официантка объяснила задержку именно большим о...,negative
3996,15567,горячие полотенца,Interior,"Несколько раз спрошенные горячие полотенца, та...",negative
3997,15567,интерьер,Interior,"Все могло бы быть неплохо: месторасположение, ...",positive
3998,15567,кухня,Food,"Все могло бы быть неплохо: месторасположение, ...",positive


Спелчекер работает долго, поэтому эксперимент проводился на маленьком датасете.

In [ ]:
small_df = train_set.contexts[:500]

In [ ]:
small_df

,id,input,aspect,context,sentiment
0,3976,ресторане,Whole,Решил написать отзыв о ресторане в котором отм...,neutral
1,3976,ресторанах,Whole,Решил написать отзыв о ресторане в котором отм...,neutral
2,3976,ресторане,Whole,Но теперь о ресторане.,neutral
3,3976,Столик бронировали,Service,Столик бронировали заранее и сделали так как п...,neutral
4,3976,администратор,Service,Столик бронировали заранее и сделали так как п...,positive
...,...,...,...,...,...
495,1105,заведению,Whole,Это не реклама этого заведения и не какой нибу...,neutral
496,32856,ресторан Марчеллис,Whole,Сегодня мы всей семьей посетили ресторан Марче...,neutral
497,32856,кухня,Food,"Были впервые, остались довольны, кухня и обслу...",positive
498,32856,обслуживание,Service,"Были впервые, остались довольны, кухня и обслу...",positive


В каждом контексте складываем (1) тональную оценку всех слов, (2) оценку только положительных и (3) только отрицательных слов.

In [ ]:
def evaluate_text(text):
    doc = Doc(text)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    summ = 0
    positive = 0
    negative = 0

    for token in doc.tokens:
        token.lemmatize(morph_vocab)

        if token.lemma in kss_terms:

            value = int(kss[kss['term'] == token.lemma].value)
            summ += value

            if value > 0.01:
                positive += value

            elif value < -0.01:
                negative += value

    length = len(doc.tokens)
    return (summ / length, positive, negative)

In [ ]:
small_df['context'] = small_df['context'].apply(spellcheck)

<ipython-input-15-82a725b3efa6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['context'] = small_df['context'].apply(spellcheck)


In [ ]:
small_df['values'] = small_df['context'].apply(evaluate_text)

<ipython-input-18-1e1598966204>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['values'] = small_df['context'].apply(evaluate_text)


In [ ]:
Меняем категории на айди.

In [ ]:
map_cats = {'neutral': 0, 'positive': 10, 'negative': 11, 'both': -10}

In [ ]:
small_df['cats'] = small_df['sentiment'].map(map_cats)

<ipython-input-20-7fad3ded1936>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['cats'] = small_df['sentiment'].map(map_cats)


In [ ]:
small_df

,id,input,aspect,context,sentiment,values,cats
0,3976,ресторане,Whole,Решил написать отзыв о ресторане в котором отм...,neutral,"(0.058823529411764705, 2, 0)",0
1,3976,ресторанах,Whole,Решил написать отзыв о ресторане в котором отм...,neutral,"(0.058823529411764705, 2, 0)",0
2,3976,ресторане,Whole,Но теперь о ресторане.,neutral,"(0.0, 0, 0)",0
3,3976,Столик бронировали,Service,Столик бронировали заранее и сделали так как п...,neutral,"(0.0, 0, 0)",0
4,3976,администратор,Service,Столик бронировали заранее и сделали так как п...,positive,"(0.0, 0, 0)",10
...,...,...,...,...,...,...,...
495,1105,заведению,Whole,Это не реклама этого заведения и не какой нибу...,neutral,"(0.0, 0, 0)",0
496,32856,ресторан Марчеллис,Whole,Сегодня мы всей семьей посетили ресторан Марче...,neutral,"(0.0, 0, 0)",0
497,32856,кухня,Food,"Были впервые, остались довольны, кухня и обслу...",positive,"(0.046153846153846156, 3, 0)",10
498,32856,обслуживание,Service,"Были впервые, остались довольны, кухня и обслу...",positive,"(0.046153846153846156, 3, 0)",10


In [ ]:
small_df['summ'] = small_df['values'].apply(lambda x: x[0])
small_df['positive'] = small_df['values'].apply(lambda x: x[1])
small_df['negative'] = small_df['values'].apply(lambda x: x[2])

#small_df = pd.get_dummies(small_df,columns=['sentiment'],drop_first=True)

<ipython-input-22-9c01e32448b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['summ'] = small_df['values'].apply(lambda x: x[0])
<ipython-input-22-9c01e32448b7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['positive'] = small_df['values'].apply(lambda x: x[1])
<ipython-input-22-9c01e32448b7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [ ]:
small_df = small_df.drop('sentiment', axis=1)
small_df = small_df.drop('input', axis=1)
small_df = small_df.drop('aspect', axis=1)
small_df = small_df.drop('context', axis=1)
small_df = small_df.drop('values', axis=1)

In [ ]:
small_df

,id,cats,summ,positive,negative
0,3976,0,0.058824,2,0
1,3976,0,0.058824,2,0
2,3976,0,0.000000,0,0
3,3976,0,0.000000,0,0
4,3976,10,0.000000,0,0
...,...,...,...,...,...
495,1105,0,0.000000,0,0
496,32856,0,0.000000,0,0
497,32856,10,0.046154,3,0
498,32856,10,0.046154,3,0


Обучаем наивный Байес.

In [ ]:
from sklearn.model_selection import train_test_split

X = small_df.drop('cats', axis=1)
y = small_df['cats']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=69
)

In [ ]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

model.fit(X_train, y_train)

GaussianNB()

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    classification_report,
)

y_pred = model.predict(X_test)

accuray = accuracy_score(y_pred, y_test)
f1 = f1_score(y_pred, y_test, average="weighted")

print("Accuracy:", accuray)
print("F1 Score:", f1)

Accuracy: 0.58
F1 Score: 0.6472014696876914
